In [35]:
import websocket
import pandas as pd
import requests
from pandas.io.json import json_normalize
import json
from code_ref import get_keys
from code_ref import findata
from code_ref import FinnhubRequestException
from code_ref import FinnhubAPIException
import time
import threading
%matplotlib inline
%load_ext autoreload
%autoreload 2
from ratelimit import limits
from ratelimiter import RateLimiter


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [36]:
keys = get_keys("/Users/augustingoudet/.secret/code_file") #Finnhub Key
api_key1 = keys ['finnhub'] #api_key
company = "KER.PA" #Company Ticker

query = findata(api_key = api_key1)


In [29]:
url = "https://finnhub.io/api/v1/"

ext1 = "?symbol={}&token={}"
ext2 = "?symbol={}&metric=management&token={}"
ext3 = "?symbol={}&metric=margin&token={}"
ext4 = "?symbol={}&metric=growth&token={}"
ext5 = "?symbol={}&metric=price&token={}"
ext6 = "?symbol={}&metric=valuation&token={}"
ext7 = "?symbol={}&metric=financialStrength&token={}"
ext8 = "?symbol={}&metric=perShare&token={}"




profile ="stock/profile"
ceo_comp = "stock/ceo-compensation"
board = "stock/executive"
reco = "stock/recommendation"
price_target = "stock/price-target"
up_down = "stock/upgrade-downgrade"
peers = "stock/peers"
rev_est = "stock/revenue-estimate"
eps_est = "stock/eps-estimate"
ownership = "stock/investor-ownership"
metric= "stock/metric"
quote = "quote"
news = "news"
covid = "covid19"
transcript = "stock/transcripts"
divs = "stock/dividend?symbol={}&from=2019-02-01&to=2020-02-01"
split = "stock/split?symbol=AAPL&from=2010-02-01&to=2020-02-01"
inc = "stock/financials?symbol={}&statement=is&freq=annual"
bs = "stock/financials?symbol={}&statement=bs&freq=annual"
cfs = "stock/financials?symbol={}&statement=cf&freq=annual"


In [30]:
tickers = pd.read_csv('Ticker_list_1.csv')

In [31]:
tick = list(tickers.Ticker)
tick1 = tick[:15]
tick1

['OEDV',
 'AAPL',
 'BAC',
 'AMZN',
 'T',
 'GOOG',
 'MO',
 'DAL',
 'AA',
 'AXP',
 'DD',
 'BABA',
 'ABT',
 'UA',
 'AMAT']

## Profiling Data

In [16]:
results = []


for items in tick1:
#     @limits(calls = 12, period = ONE_MINUTE)
    data = query.ceo_compensation(symbol = items)
    results.append(data)
    df = pd.DataFrame(results)
    df.dropna(inplace = True)
    time.sleep(65)

%time

CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 8.82 µs


In [17]:
df

,bonus,companyName,location,name,nonEquityIncentives,optionAwards,otherComp,pensionAndDeferred,salary,stockAwards,symbol,total,year
1,0.0,Apple Inc.,"Cupertino, CA",Timothy Cook,12000000.0,0.0,682219.0,0.0,3000000.0,0.0,AAPL,15682219.0,2018
2,0.0,Bank of America Corp,"Charlotte, NC",Brian Moynihan,0.0,0.0,382263.0,320159.0,1500000.0,20552088.0,BAC,22754510.0,2018


## board members / executive

In [124]:
results = []

for items in tick1: 
    data = url + ceo_comp + ext1.format(items, api_key1)
    req = requests.get(data)
    content = req.json()
    results.append(content)
    df1 = pd.DataFrame(results)
    df1.dropna(inplace = True)

In [125]:
df1

,bonus,companyName,location,name,nonEquityIncentives,optionAwards,otherComp,pensionAndDeferred,salary,stockAwards,symbol,total,year
0,0.0,DuPont de Nemours Inc.,"Wilimington, DE",Edward Breen,3300510.0,12700004.0,743988.0,0.0,1930800.0,0.0,DD,18675302.0,2018
2,0.0,Abbott Laboratories,"Abbott Park, IL",Miles White,4779688.0,7499996.0,1193342.0,1381845.0,1900000.0,7499367.0,ABT,24254238.0,2018
4,0.0,Applied Materials Inc.,"Santa Clara, CA",Gary Dickerson,1430000.0,0.0,373229.0,0.0,1000000.0,11261311.0,AMAT,14064540.0,2018
5,0.0,Amgen Inc.,"Thousand Oaks, CA",Robert Bradway,3898000.0,3749994.0,591454.0,0.0,1566000.0,8749818.0,AMGN,18555266.0,2018
6,0.0,American Airlines Group Inc.,"Fort Worth, TX",W. Parker,0.0,0.0,60517.0,0.0,0.0,11939000.0,AAL,11999517.0,2018
7,0.0,American International Group Inc.,"New York, NY",Brian Duperreault,3040000.0,4199993.0,257487.0,0.0,1600000.0,11757189.0,AIG,20854669.0,2018
8,0.0,Allstate Corporation,"Northbrook, IL",Thomas Wilson,6719194.0,3874998.0,116971.0,873170.0,1290385.0,5812528.0,ALL,18687246.0,2018
9,0.0,Adobe Inc.,"San Jose, CA",Shantanu Narayen,1824313.0,0.0,33451.0,0.0,1000000.0,25539764.0,ADBE,28397528.0,2018


## analyst recommendation 

In [22]:
results = []

for items in tick1[:3]: 
    data = query.earnings(symbol = items)
    eps = pd.DataFrame(data)

In [23]:
eps

,actual,estimate,period,symbol
0,0.95,0.94600,2019-12-31,ABT
1,0.84,0.84139,2019-09-30,ABT
2,0.82,0.80220,2019-06-30,ABT
3,0.63,0.61270,2019-03-31,ABT
4,0.81,0.80986,2018-12-31,ABT
...,...,...,...,...
70,0.49,0.49471,2002-06-30,ABT
71,0.54,0.54056,2002-03-31,ABT
72,0.52,0.51250,2001-12-31,ABT
73,0.44,0.44050,2001-09-30,ABT


## Metrics

In [32]:
results = []


for items in tick1:
    with rate_limiter:
        data = url + metric + ext2.format(items, api_key1)
        req = requests.get(data).json()
        results.append(req)
        m1 = json_normalize(results)
        m1.dropna(inplace=True)

In [33]:
m1

,metricType,symbol,metric.assetTurnoverAnnual,metric.assetTurnoverTTM,metric.inventoryTurnoverAnnual,metric.inventoryTurnoverTTM,metric.netIncomeEmployeeAnnual,metric.netIncomeEmployeeTTM,metric.receivablesTurnoverAnnual,metric.receivablesTurnoverTTM,...,metric.revenueEmployeeTTM,metric.roaRfy,metric.roaa5Y,metric.roae5Y,metric.roaeTTM,metric.roeRfy,metric.roeTTM,metric.roi5Y,metric.roiAnnual,metric.roiTTM
1,management,AAPL,0.73888,0.74946,40.1345,36.5669,410825,419905,11.2844,13.7108,...,1953891,15.69236,16.0985,44.7147,55.46829,55.91722,16.10640,22.4014,22.8996,22.833
2,management,BAC,,,,,133155,,,,...,,1.14564,0.97437,8.51805,10.73393,10.73393,1.14564,,,
3,management,AMZN,1.44638,1.44637,8.78851,8.78851,16052.6,14538.9,15.0749,15.0749,...,351530,5.98202,4.61216,19.0376,21.94510,21.94510,5.98202,8.38666,10.015,10.015
4,management,T,0.33445,0.33445,29.8637,,58268.5,60874,6.97392,6.97392,...,736557,2.76411,3.20867,9.61561,7.54799,7.54799,2.76411,3.75153,3.24593,3.24593
6,management,MO,0.47952,0.47952,5.36202,5.36202,-155641,-166027,170.816,170.816,...,3439726,-2.31834,14.8161,63.8761,-11.55695,-11.57599,-2.31452,19.2954,-3.22928,-3.22396
7,management,DAL,0.75333,0.75333,18.3304,18.333,52994.9,52245,18.1916,18.1916,...,515292,7.63955,7.57275,34.2158,32.81804,32.82493,7.63794,11.1361,11.084,11.0817
8,management,AA,0.67828,0.67828,4.93041,4.93041,-61366.9,-61811.6,15.1642,15.1642,...,756014,-5.54562,-0.52405,-5.70377,-23.12436,-23.12436,-5.54562,-0.74243,-7.92125,-7.92125
9,management,AXP,0.2332,0.2332,,,,,0.80481,0.80481,...,,3.49372,3.34681,27.494,29.23657,29.23657,3.49372,6.10306,6.53072,6.46358
10,management,DD,0.16725,0.16725,3.33634,3.33634,-10210.5,-19400,5.91314,5.95241,...,614629,-0.52789,,,-1.04968,-1.04968,-0.52789,,-0.78309,-0.78309
11,management,BABA,0.44804,0.43701,31.2321,,133390,237562,20.6038,,...,675913,9.47188,12.891,22.942,28.53605,20.41770,15.35962,17.8298,13.8339,22.1577


In [141]:
results = []

for items in tick1: 
    with rate_limiter:
        data = url + metric + ext3.format(items, api_key1)
        req = requests.get(data).json()
        results.append(req)
        m2 = json_normalize(results)
        m2.dropna(inplace=True)

In [142]:
m2

,metricType,symbol,metric.freeOperatingCashFlow/revenue5Y,metric.freeOperatingCashFlow/revenueTTM,metric.grossMargin5Y,metric.grossMarginAnnual,metric.grossMarginTTM,metric.netProfitMargin%Annual,metric.netProfitMargin5Y,metric.netProfitMarginTTM,metric.operatingMargin5Y,metric.operatingMarginAnnual,metric.operatingMarginTTM,metric.pretaxMargin5Y,metric.pretaxMarginAnnual,metric.pretaxMarginTTM
0,margin,DD,,-46.43455,,34.6597,34.6597,-3.15638,,-3.15638,,-0.30681,-0.30681,,-2.20342,-2.20342
1,margin,BABA,38.7354,40.87565,55.632,45.8434,45.699,21.14084,31.8941,35.14681,23.0066,15.14791,19.03558,38.4788,25.53338,39.84291
2,margin,ABT,6.63616,6.98345,57.2195,58.6384,58.6384,11.28699,8.80577,11.28699,11.7509,14.00765,14.00765,10.5064,12.77896,12.77896
4,margin,AMAT,14.947,14.77659,43.5792,43.7158,43.8103,18.35980,20.2256,18.82533,23.4935,22.93264,23.20037,22.6964,22.37815,22.54778
5,margin,AMGN,28.3411,21.50073,81.7513,81.3543,81.3543,33.56733,34.0072,33.56733,42.1024,41.40913,41.40913,39.6306,39.11480,39.11480
6,margin,AAL,-0.96467,-1.37869,63.4075,61.4643,61.4622,3.68380,7.19274,3.68380,9.9111,6.69682,6.69682,7.61598,4.92921,4.92702
7,margin,AIG,-3.16081,-4.26082,,,,8.25521,2.6351,8.25521,6.42738,13.42949,13.42949,3.94358,10.59095,10.59095
8,margin,ALL,8.74117,8.74047,,,,10.83782,7.14061,10.83782,9.79519,13.61492,13.61492,9.79519,13.61492,13.61492
9,margin,ADBE,36.1842,36.96102,85.7351,85.0266,85.1867,26.44646,24.1751,27.74269,27.9751,29.25462,30.10192,27.3773,28.68728,29.33971


In [143]:
results = []

for items in tick1: 
    with rate_limiter:
        data = url + metric + ext4.format(items, api_key1)
        req = requests.get(data).json()
        results.append(req)
        m3 = json_normalize(results)
        m3.dropna(inplace=True)

In [144]:
m3

,metricType,symbol,metric.bookValueShareGrowth5Y,metric.capitalSpendingGrowth5Y,metric.dividendGrowthRate5Y,metric.ebitdaCagr5Y,metric.ebitdaInterimCagr5Y,metric.epsGrowth3Y,metric.epsGrowth5Y,metric.epsGrowthQuarterlyYoy,metric.epsGrowthTTMYoy,metric.focfCagr5Y,metric.netMarginGrowth5Y,metric.revenueGrowth3Y,metric.revenueGrowth5Y,metric.revenueGrowthQuarterlyYoy,metric.revenueGrowthTTMYoy,metric.revenueShareGrowth5Y,metric.tbvCagr5Y,metric.totalDebtCagr5Y
0,growth,DD,,,-26.8575,,,,,-50.39317,-353.82900,,,-23.55703,,-4.63625,-4.78888,,,
1,growth,BABA,60.9776,49.3443,,29.5595,38.2833,6.18768,27.5425,54.70443,145.60850,40.1034,-14.0079,55.02738,48.318,37.66947,41.59460,44.8705,44.8181,26.7359
2,growth,ABT,4.42139,8.74759,7.16645,12.254,13.4695,41.2936,12.3992,41.59599,45.24415,12.1463,5.83555,15.22821,9.52087,7.07019,4.33645,6.20181,,18.9592
4,growth,AMAT,6.83218,12.8455,27.0334,17.3862,14.9995,22.5474,26.6537,24.30596,-18.54218,13.635,9.21323,10.50644,9.99616,10.89795,-7.60475,15.9692,5.21677,22.2348
5,growth,AMGN,-13.564,-9.23101,13.1851,6.61116,5.59962,7.93203,13.963,-5.15992,2.17905,-2.50049,5.47906,0.53502,3.09149,-0.52970,-1.62126,8.04322,,-0.53441
6,growth,AAL,,-4.27847,0,-0.86016,-2.0735,-6.52604,-0.67816,35.08849,25.10292,,-11.4264,4.46904,1.42116,3.42841,2.75477,12.1348,,6.14596
7,growth,AIG,-0.57367,,0,-5.10986,-5.10986,,-6.7828,239.39090,15242.70000,,-6.76951,-1.88571,-5.03196,-6.33051,4.78589,4.6825,-9.28354,2.51789
8,growth,ALL,8.84001,8.49734,14.9623,6.03153,6.35797,44.3103,17.4863,406.41010,150.56380,10.3956,6.17411,6.67015,4.73866,20.97828,12.29035,10.6171,2.15155,5.22603
9,growth,ADBE,9.89523,21.6068,,41.117,38.4436,37.3871,62.6583,43.43712,19.62977,28.7323,32.5155,24.03366,21.9193,18.84138,22.08273,22.7467,,22.2689


In [14]:
results = []

for items in tick1: 
    with rate_limiter:
        data = url + metric + ext5.format(items, api_key1)
        req = requests.get(data).json()
        results.append(req)
        m4 = json_normalize(results)
        m4.dropna(inplace=True)

In [15]:
m4

,metricType,symbol,metric.10DayAverageTradingVolume,metric.13WeekPriceReturnDaily,metric.26WeekPriceReturnDaily,metric.3MonthAverageTradingVolume,metric.52WeekHigh,metric.52WeekHighDate,metric.52WeekLow,metric.52WeekLowDate,...,metric.5DayPriceReturnDaily,metric.beta,metric.marketCapitalization,metric.monthToDatePriceReturnDaily,metric.priceRelativeToS&P50013Week,metric.priceRelativeToS&P50026Week,metric.priceRelativeToS&P5004Week,metric.priceRelativeToS&P50052Week,metric.priceRelativeToS&P500Ytd,metric.yearToDatePriceReturnDaily
0,price,DD,7.99046,-46.88474,-49.81604,170.81221,83.7215,2019-04-17,28.3300,2020-03-23,...,5.05237,,25213.150,-20.51282,-33.60505,-42.91033,-9.19980,-51.14667,-33.60505,-46.88474
1,price,BABA,20.16349,-12.47505,13.62212,352.85681,231.1400,2020-01-13,147.9500,2019-05-31,...,4.02096,1.80061,505934.800,-9.33173,11.58211,32.41347,5.39334,17.54907,13.03173,-11.08439
2,price,ABT,16.02029,-8.59447,-5.17509,170.68322,92.4500,2020-01-22,61.6100,2020-03-23,...,26.29736,0.94944,139910.800,2.99883,12.09857,7.46354,14.37666,7.09939,12.35116,-8.65761
4,price,AMAT,13.56253,-22.36993,-5.21042,201.24300,69.4400,2020-02-13,36.6400,2020-03-18,...,17.74956,1.49668,43435.830,-18.61666,-4.79548,7.42350,-7.49996,28.69669,-4.68708,-22.50983
5,price,AMGN,4.95217,-13.23095,7.73603,66.49391,244.9900,2019-12-17,166.3000,2019-05-15,...,12.12822,0.9046,122962.900,4.38091,6.41246,22.09551,15.58663,18.41738,6.37178,-13.51889
6,price,AAL,73.58842,-50.63291,-48.01925,569.39985,35.2400,2019-04-15,10.0100,2020-03-23,...,35.26012,1.86276,5981.865,-26.29921,-37.06384,-39.42244,-14.32976,-49.66502,-37.76849,-51.04603
7,price,AIG,11.78008,-49.58824,-53.84201,156.18375,58.6600,2019-09-19,16.0700,2020-03-18,...,33.21244,1.48485,22455.680,-39.01803,-38.17566,-47.68990,-29.41054,-35.57017,-38.39216,-49.91233
8,price,ALL,3.18122,-15.32612,-12.91866,44.48290,125.9200,2020-02-18,64.1300,2020-03-18,...,29.23665,0.90523,29992.710,-10.08076,3.84297,-1.31212,-1.17768,8.43637,3.51909,-15.83815
9,price,ADBE,6.09441,-7.54557,10.19313,73.49865,386.7450,2020-02-20,255.1314,2020-03-18,...,3.55184,0.96782,147349.200,-11.38445,17.86651,28.41737,3.00725,28.56161,17.87960,-7.27085


In [26]:
# results = []

# for items in tick1: 
#     with rate_limiter:
#         data = url + metric + ext6.format(items, api_key1)
#         req = requests.get(data).json()
#         results.append(req)
#         m5 = json_normalize(results)
#         m5.dropna(inplace=True)
# m5

In [24]:
# results = []

# for items in tick1: 
#     with rate_limiter:
#         data = url + metric + ext7.format(items, api_key1)
#         req = requests.get(data).json()
#         results.append(req)
#         m6 = json_normalize(results)
#         m6.dropna(inplace=True)
# m6

In [25]:
# results = []

# for items in tick1: 
#     with rate_limiter:
#         data = url + metric + ext8.format(items, api_key1)
#         req = requests.get(data).json()
#         results.append(req)
#         m7 = json_normalize(results)
#         m7.dropna(inplace=True)
    
# m7

## Stock Upgrade / Downgrade

In [6]:
url4 = "https://finnhub.io/api/v1/stock/upgrade-downgrade?symbol={}&token={}".format(company, api_key1) #url
req4 = requests.get(url4) 
status = pd.DataFrame(req4.json())

In [7]:
status

,symbol,gradeTime,company,fromGrade,toGrade,action
0,KER.PA,1541116800,RBC Capital,Sector Perform,Outperform,up


### Peers

In [ ]:
url5 = "https://finnhub.io/api/v1/stock/peers?symbol={}&token={}".format(company, api_key) #url
req5 = requests.get(url5) 

In [ ]:
print(req5.json())

## Revenue Estimates

In [ ]:
url6 = "https://finnhub.io/api/v1/stock/revenue-estimate?symbol={}&token={}".format(company, api_key) #url
req6 = requests.get(url6) 
rev = pd.DataFrame(req6.json())
rev1 = json_normalize(rev['data'])
rev1

## Earnings Estimate

In [ ]:
url7 = "https://finnhub.io/api/v1/stock/eps-estimate?symbol={}&token={}".format(company, api_key) #url
req7 = requests.get(url7) 
print(req7.json())

In [ ]:
eps = pd.DataFrame(req7.json())
eps1 = json_normalize(eps['data'])
eps1

## Investor Ownership

In [ ]:
url8 = "https://finnhub.io/api/v1/stock/investor-ownership?symbol={}&token={}".format(company, api_key) #url
req8 = requests.get(url8) 
# print(req8.json())
own = req8.json()

In [ ]:
ownership = pd.DataFrame(own.items(), columns =['Features', 'Value'])
# print(req8.json())
owner = pd.DataFrame(req8.json())
owner1 = json_normalize(owner['ownership'])

In [ ]:
owner1

## Fund Ownership

In [ ]:
url9 = "https://finnhub.io/api/v1/stock/fund-ownership?symbol={}&token={}".format(company, api_key) #url
req9 = requests.get(url9)
fdowner = pd.DataFrame(req9.json())
fdowner1 = json_normalize(fdowner['ownership'])
fdowner1

## stock quote

In [ ]:
url10 = "https://finnhub.io/api/v1/quote?symbol={}&token={}".format(company, api_key) #url
req10 = requests.get(url10) 
print(req10.json())

In [ ]:
url10 = "https://finnhub.io/api/v1/quote?symbol=BINANCE:BTCEUR&token={}".format(api_key) #url
req10 = requests.get(url10) 
print(req10.json())

## General News

In [ ]:
url11 = "https://finnhub.io/api/v1/news?category=general&token={}".format(api_key) #url
req11 = requests.get(url11) 
news = pd.DataFrame(req11.json())
news

## company news 

In [ ]:
url12 = "https://finnhub.io/api/v1/news/{}?token={}".format(company,api_key) #url
req12 = requests.get(url12) 
compnews = pd.DataFrame(req12.json())
compnews

## Major Developments

In [ ]:
url13 = "https://finnhub.io/api/v1/major-development?symbol={}&token={}'".format(company,api_key) #url
req13 = requests.get(url13) 
# print(req13.json())

## New Sentiment

In [ ]:
url14 = "https://finnhub.io/api/v1/news-sentiment?symbol={}&token={}".format(company,api_key) #url
req14 = requests.get(url14) 

cpn = pd.DataFrame(req14.json())
print(req14.json())

## Economic Code & Data


In [ ]:
url15 = "https://finnhub.io/api/v1/economic/code?token={}".format(api_key) #url
req15 = requests.get(url15) 
# print(req15.json())


In [ ]:
code = pd.DataFrame(req15.json())
code.head(n=15)

In [ ]:
df = code.loc[code['country'] == 'United States']

In [ ]:
code.code.value_counts().sum()

In [ ]:
url16 = "https://finnhub.io/api/v1/economic?code=A-FRA-6682&token={}".format(api_key) #url
req16 = requests.get(url16) 
eco = pd.DataFrame(req16.json(), columns =['Date', 'Monthly Bankruptices in France'])
eco

In [ ]:
url17 = "https://finnhub.io/api/v1/covid19/us?token={}".format(api_key) #url
req17 = requests.get(url17) 
# print(req17.json())

covid = pd.DataFrame(req17.json())
covid.head(n=20)

## Financials

In [48]:
import requests
r = requests.get('https://finnhub.io/api/v1/stock/financials?symbol=AAPL&statement=bs&freq=annual&token=bq06smvrh5rddd65esl0')
print(r.json())


{'financials': [{'accountsPayable': 46236, 'accountsReceivableTradeNet': 22926, 'accruedExpenses': 20951, 'accumulatedDepreciationTotal': -58579, 'cash': 12204, 'cashEquivalents': 36640, 'cashandShortTermInvestments': 100557, 'commonStockTotal': 45174, 'currentPortofLTDebtCapitalLeases': 10260, 'deferredIncomeTax': 31504, 'goodwillNet': 5414, 'intangiblesNet': 3206, 'longTermDebt': 91807, 'longTermInvestments': 105341, 'notesPayableShortTermDebt': 5980, 'otherCurrentAssetsTotal': 12352, 'otherCurrentliabilitiesTotal': 43242, 'otherEquityTotal': -1291, 'otherLiabilitiesTotal': 50503, 'otherLongTermAssetsTotal': 32978, 'period': '2019-09-28', 'propertyPlantEquipmentTotalGross': 95957, 'propertyPlantEquipmentTotalNet': 37378, 'retainedEarningsAccumulatedDeficit': 45898, 'shortTermInvestments': 51713, 'tangibleBookValueperShare': 20.36534, 'totalAssets': 338516, 'totalCommonSharesOutstanding': 4443.236, 'totalCurrentAssets': 162819, 'totalCurrentLiabilities': 105718, 'totalDebt': 108047, '

In [54]:
df = pd.DataFrame(r.json())

In [56]:
df1 = json_normalize(df['financials'])

In [58]:


df1.columns

Index(['accountsPayable', 'accountsReceivableTradeNet', 'accruedExpenses',
       'accumulatedDepreciationTotal', 'cash', 'cashEquivalents',
       'cashandShortTermInvestments', 'commonStockTotal',
       'currentPortofLTDebtCapitalLeases', 'deferredIncomeTax', 'goodwillNet',
       'intangiblesNet', 'longTermDebt', 'longTermInvestments',
       'notesPayableShortTermDebt', 'otherCurrentAssetsTotal',
       'otherCurrentliabilitiesTotal', 'otherEquityTotal',
       'otherLiabilitiesTotal', 'otherLongTermAssetsTotal', 'period',
       'propertyPlantEquipmentTotalGross', 'propertyPlantEquipmentTotalNet',
       'retainedEarningsAccumulatedDeficit', 'shortTermInvestments',
       'tangibleBookValueperShare', 'totalAssets',
       'totalCommonSharesOutstanding', 'totalCurrentAssets',
       'totalCurrentLiabilities', 'totalDebt', 'totalEquity', 'totalInventory',
       'totalLiabilities', 'totalLiabilitiesShareholdersEquity',
       'totalLongTermDebt', 'totalReceivablesNet', 'unrealizedG